In [ ]:
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib as plot
import math
from scipy import stats

crime_housing = pd.read_csv('crime-housing-austin-2015.csv')
crime_housing.head()
print(crime_housing.columns)

In [ ]:
# build df with relevant cols. 
df_poverty = pd.read_csv('crime-housing-austin-2015.csv', usecols=[
    'Highest_NIBRS_UCR_Offense_Description', 
    'Zip_Code_Crime',
    'Changeinpercentageofpopulationbelowpoverty2000-2012',
    'Populationbelowpovertylevel'
    ])

df_poverty = df_poverty.rename(columns={'Zip_Code_Crime': 'Zip Code'})







In [ ]:
df_zip_codes = pd.read_csv('AustinZipCodes.csv', usecols=['Zip Code', 'Population', 'People / Sq. Mile'])
# merged_df = pd.merge(df_poverty, df_zip_codes, left_on='Zip_Code_Crime', right_on='Zip Code', how='left')


# Merge data frames on the 'Zip Code' column
merged_df = pd.merge(df_poverty, df_zip_codes, on='Zip Code', how='left')

# Calculate total number of crime reports for each zip code
crime_reports_count = merged_df['Zip Code'].value_counts().reset_index()

crime_reports_count.columns = ['Zip Code', 'Total_Crimes']

# Merge the crime reports count with df_zip_codes
df_zip_codes = pd.merge(df_zip_codes, crime_reports_count, on='Zip Code', how='left')

# Fill NaN values with 0 for zip codes with no crime reports
df_zip_codes['Total_Crimes'].fillna(0, inplace=True)

df_zip_codes['People / Sq. Mile'] = df_zip_codes['People / Sq. Mile'].str.replace(',','').astype('float64')
df_zip_codes['Population']= df_zip_codes['Population'].str.replace(',','').astype('float64')
# df_zip_codes = df_zip_codes.dropna()
# print(df_zip_codes['Population'].dtype)

# add crimes per capita column.
df_zip_codes['Crimes_Per_Capita'] = (df_zip_codes['Total_Crimes'] / df_zip_codes['Population'])

# add percentage of pop below poverty to 
percent_poverty = df_poverty[['Zip Code', 'Populationbelowpovertylevel']].drop_duplicates().reset_index(drop=True)
percent_change_poverty = df_poverty[['Zip Code', 'Changeinpercentageofpopulationbelowpoverty2000-2012']].drop_duplicates().reset_index(drop=True)
df_zip_codes = pd.merge(df_zip_codes, percent_poverty, on='Zip Code', how='left')
df_zip_codes = pd.merge(df_zip_codes, percent_change_poverty, on='Zip Code', how='left')

df_zip_codes.head()

In [ ]:
dropped = df_zip_codes.dropna()
dropped['Populationbelowpovertylevel'] = dropped['Populationbelowpovertylevel'].str.replace('%','').astype('float64')
dropped['Changeinpercentageofpopulationbelowpoverty2000-2012'] = dropped['Changeinpercentageofpopulationbelowpoverty2000-2012'].str.replace('%','').astype('float64')
stats.pearsonr(dropped['Total_Crimes'], dropped['Populationbelowpovertylevel'])

In [ ]:
sb.scatterplot(x=df_zip_codes['People / Sq. Mile'], y=df_zip_codes['Crimes_Per_Capita'])